In [1]:
import pandas as pd
import numpy as np
import surprise
import apyori

In [2]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

In [3]:
trainingSet.n_users

943

In [4]:
testSet = trainingSet.build_testset()

In [31]:
from surprise import KNNBasic

knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()

In [32]:
knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [33]:
testSet = trainingSet.build_testset()
predictions = knn.test(testSet)

In [17]:
predictions[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=3.6206349533418014, details={'actual_k': 39, 'was_impossible': False})

Задание 9.10.4

Рассчитайте метрику качества RMSE для предсказаний через функцию surprise.accuracy.rmse() для обоих обученных ранее алгоритмов. В качестве ответа укажите значение RMSE для того алгоритма, который дал лучший результат. Округлите ответ до двух знаков после точки-разделителя.

In [34]:
surprise.accuracy.rmse(predictions)

RMSE: 0.8931


0.8930657721155663

In [35]:
from surprise import SVD

In [36]:
svd = SVD()
trainingSet = data.build_full_trainset()
svd.fit(trainingSet)

In [37]:
testSet = trainingSet.build_testset()
predictions = svd.test(testSet)

In [41]:
surprise.accuracy.rmse(predictions)

RMSE: 0.6746


0.6746275814091125

In [42]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


Задание 9.10.5

Сформируйте для каждого user_id текстовую строку, содержащую набор movie_id, получивших оценку 4 и выше. Используйте пробел (" ") в качестве разделителя.

Для проверки результата введите строку для user_id 100.

In [46]:
good = df[df['rating']>=4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [49]:
good

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
                             ...                        
95     625 233 68 1 527 172 1206 416 174 143 96 843 1...
96     156 87 673 479 153 91 519 8 484 645 64 100 170...
97     228 222 670 89 482 98 50 115 7 435 28 428 655 ...
98     47 517 116 629 523 514 25 428 435 211 210 659 194
99     4 79 1016 403 50 742 181 182 597 410 168 204 5...
Name: movie_id, Length: 942, dtype: object

In [51]:
good.loc['100']

'344 355 750 302 691 316 752 313 879 300 328 1235 690 347 269 258 900 294 272 751 898 315'

Задание 9.10.6

Используя алгоритм apriory из модуля apyory рассчитайте ассоциативные правила для полученного набора фильмов с положительными оценками. Будем считать, что хороший рейтинг - это когда оценка >=4. Используйте следующие ограничения:

минимальный support 0.2,
минимальное значение confidence 0.3,
минимальное значение lift 2,
минимальная длина 2.
Сколько правил получилось?

In [52]:
import apyori

In [58]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.2, 
                                   min_confidence=0.3, min_lift=2, 
                                   min_length=2)

In [59]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [60]:
asr_df

,from,to,confidence,support,lift
0,172,174,0.819113,0.254777,2.217253
1,173,174,0.762097,0.200637,2.062917
2,174,195,0.548851,0.20276,2.382568
3,174,204,0.543103,0.200637,2.177036
4,174,210,0.563218,0.208068,2.257667
5,174,79,0.591954,0.218684,2.1122
6,172,174 181,0.651877,0.20276,2.658304
7,172,174 50,0.78157,0.2431,2.512761
8,172,50 181,0.750853,0.233546,2.026658
9,174,98 50,0.577586,0.213376,2.125337
